mnist分類学習（CPU）
chainer公式exampleより改変
https://github.com/chainer/chainer

In [17]:
import argparse
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions

In [18]:
# Network definition
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)  # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

In [33]:
# 引数
class Args(object):
    def __init__(self):
        self.batchsize = 100
        self.epoch = 2
        self.frequency = -1
        self.gpu = -1
        self.out = "result-chainer-mnist"
        self.resume = ' '
        self.unit = 50

In [34]:
args = Args()
print('GPU: {}'.format(args.gpu))
print('# unit: {}'.format(args.unit))
print('# Minibatch-size: {}'.format(args.batchsize))
print('# epoch: {}'.format(args.epoch))
print('')

GPU: -1
# unit: 50
# Minibatch-size: 100
# epoch: 2



In [35]:
# Set up a neural network to train
# Classifier reports softmax cross entropy loss and accuracy at every
# iteration, which will be used by the PrintReport extension below.
model = L.Classifier(MLP(args.unit, 10))

In [36]:
# Setup an optimizer
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [37]:
# Load the MNIST dataset
train, test = chainer.datasets.get_mnist()

train_iter = chainer.iterators.SerialIterator(train, args.batchsize)
test_iter = chainer.iterators.SerialIterator(test, args.batchsize, repeat=False, shuffle=False)

In [38]:
# Set up a trainer
updater = training.updaters.StandardUpdater(train_iter, optimizer, device=args.gpu)
trainer = training.Trainer(updater, (args.epoch, 'epoch'), out=args.out)

In [39]:
# Evaluate the model with the test dataset for each epoch
trainer.extend(extensions.Evaluator(test_iter, model, device=args.gpu))

In [40]:
# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

In [41]:
# Take a snapshot for each specified epoch
frequency = args.epoch if args.frequency == -1 else max(1, args.frequency)
trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))

In [42]:
# Write a log of evaluation statistics for each epoch
trainer.extend(extensions.LogReport())

In [43]:
trainer.extend(
    extensions.PlotReport(['main/loss', 'validation/main/loss'],
                          'epoch', file_name='loss.png'))
trainer.extend(
    extensions.PlotReport(
        ['main/accuracy', 'validation/main/accuracy'],
        'epoch', file_name='accuracy.png'))

In [44]:
# Print selected entries of the log to stdout
# Here "main" refers to the target link of the "main" optimizer again, and
# "validation" refers to the default name of the Evaluator extension.
# Entries other than 'epoch' are reported by the Classifier link, called by
# either the updater or the evaluator.
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

In [45]:
# Print a progress bar to stdout
trainer.extend(extensions.ProgressBar())

In [46]:
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
     total [####..............................................]  8.33%
this epoch [########..........................................] 16.67%
       100 iter, 0 epoch / 2 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [########..........................................] 16.67%
this epoch [################..................................] 33.33%
       200 iter, 0 epoch / 2 epochs
    287.38 iters/sec. Estimated time to finish: 0:00:03.479753.
     total [############......................................] 25.00%
this epoch [#########################.........................] 50.00%
       300 iter, 0 epoch / 2 epochs
    287.01 iters/sec. Estimated time to finish: 0:00:03.135739.
     total [################..................................] 33.33%
this epoch [#################################.................] 66.67%
       400 iter, 0 epoch / 2 epochs
   